# Gcode 72

In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 


In [2]:
model = AbstractModel()
model.gen=Set()
model.t=Set()
model.a=Param(model.gen)
model.b=Param(model.gen)
model.c=Param(model.gen)
model.d=Param(model.gen)
model.e=Param(model.gen)
model.f=Param(model.gen)
model.Pmin=Param(model.gen)
model.Pmax=Param(model.gen)
model.RU=Param(model.gen)
model.RD=Param(model.gen)

model.teta=Param(model.t)
model.wind=Param(model.t)
model.demand=Param(model.t)

#Scalar
model.soco=Param(initialize=100)
model.socm=Param(initialize=300)
model.etac=Param(initialize=0.95)
model.etad=Param(initialize=0.90)

#Scalar
model.vwc=Param(initialize=50,mutable=True)

In [3]:
#constraint
def Pbounds(model,gen,t):
    return(model.Pmin[gen],model.Pmax[gen])

def Pwbounds(model,t):
      return(0,model.wind[t])

def Sbounds(model,t):
    return (0.2 * model.soco, model.socm)

def pcbounds(model,t):
    return (0,0.2*model.socm)

def pdbounds(model,t):
    return (0,0.2*model.socm)

model.p=Var(model.gen,model.t,bounds=Pbounds,within=Reals)
model.pw=Var(model.t,bounds=Pwbounds,within=Reals)
model.pwc=Var(model.t,bounds=Pwbounds,within=Reals)
model.cost=Var(within=Reals)
model.soc=Var(model.t,bounds=Sbounds,within=Reals)
model.pd=Var(model.t,bounds=pdbounds,initialize=0)
model.pc=Var(model.t,bounds=pcbounds,initialize=0)


def rule1(model,t):
    return sum(model.p[gen,t] for gen in model.gen) + model.pd[t] + model.pw[t] >= model.demand[t] + model.pc[t]
model.c1 = Constraint(model.t,rule=rule1)

def rule2(model,gen,t):
    if t==1:
        return model.p[gen,t+1] - model.p[gen,t] <= model.RU
    else:
        return Constraint.Skip
model.c2 = Constraint(model.gen,model.t,rule=rule2)

def rule3(model,gen,t):
    if t==1:
        return model.p[gen,t-1]- model.p[gen,t] <= model.RD
    else:
        return Constraint.Skip
model.c3 = Constraint(model.gen,model.t,rule=rule3)

def rule6(model, t):
    if t == 't1':
        return model.soc[t] == model.soco + model.etac * model.pc[t] - model.pd[t] / model.etad
    else:
        previous_t = 't' + str(int(t[1:]) - 1)
        return model.soc[t] == model.soc[previous_t] + model.etac * model.pc[t] - model.pd[t] / model.etad
model.c6 = Constraint(model.t, rule=rule6)

# Add a constraint to ensure model.soc[t24] equals 100
def rule_final_soc(model):
    return model.soc['t24'] == 100
model.c_final_soc = Constraint(rule=rule_final_soc)

def rule7(model,t):
    return model.pw[t] + model.pwc[t] == model.wind[t]
model.c7 = Constraint(model.t,rule=rule7)

def ruleTC(model):
    return model.cost == sum((model.a[gen]*model.p[gen,t]*model.p[gen,t] + 
                              model.b[gen]*model.p[gen,t]+model.c[gen]) for gen in model.gen for t in model.t)  + sum(model.vwc + model.pwc[t] for t in model.t)
model.c4 = Constraint(rule=ruleTC)



In [4]:
model.obj=Objective(expr=model.cost,sense=minimize)

In [5]:
instance= model.create_instance("code44.dat")

In [6]:
results = SolverFactory('ipopt',executable='C:\\ipopt\\bin\\ipopt.exe').solve(instance).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 74
  Number of variables: 217
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.10992121696472168
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [7]:
instance.cost.pprint()

cost : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 223934.27016268106 :  None : False : False :  Reals


In [8]:
instance.p.pprint()

p : Size=96, Index=p_index
    Key           : Lower : Value              : Upper : Fixed : Stale : Domain
     ('G1', 't1') :    28 : 144.48509798932136 :   200 : False : False :  Reals
    ('G1', 't10') :    28 :  132.4132373162609 :   200 : False : False :  Reals
    ('G1', 't11') :    28 : 106.87507162110346 :   200 : False : False :  Reals
    ('G1', 't12') :    28 : 119.01864766214885 :   200 : False : False :  Reals
    ('G1', 't13') :    28 :  88.54815232052584 :   200 : False : False :  Reals
    ('G1', 't14') :    28 :  79.52812110301176 :   200 : False : False :  Reals
    ('G1', 't15') :    28 :  68.88257151914735 :   200 : False : False :  Reals
    ('G1', 't16') :    28 :  73.75912558462215 :   200 : False : False :  Reals
    ('G1', 't17') :    28 :  80.02449425845549 :   200 : False : False :  Reals
    ('G1', 't18') :    28 : 108.30935202543452 :   200 : False : False :  Reals
    ('G1', 't19') :    28 : 104.05398938574828 :   200 : False : False :  Reals
     ('G1', '

In [9]:
instance.pw.pprint()

pw : Size=24, Index=t
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     t1 :     0 :  44.1 :  44.1 : False : False :  Reals
    t10 :     0 : 306.9 : 306.9 : False : False :  Reals
    t11 :     0 : 424.1 : 424.1 : False : False :  Reals
    t12 :     0 : 398.0 :   398 : False : False :  Reals
    t13 :     0 : 487.6 : 487.6 : False : False :  Reals
    t14 :     0 : 521.9 : 521.9 : False : False :  Reals
    t15 :     0 : 541.3 : 541.3 : False : False :  Reals
    t16 :     0 : 560.0 :   560 : False : False :  Reals
    t17 :     0 : 486.8 : 486.8 : False : False :  Reals
    t18 :     0 : 372.6 : 372.6 : False : False :  Reals
    t19 :     0 : 367.4 : 367.4 : False : False :  Reals
     t2 :     0 :  48.5 :  48.5 : False : False :  Reals
    t20 :     0 : 314.3 : 314.3 : False : False :  Reals
    t21 :     0 : 316.6 : 316.6 : False : False :  Reals
    t22 :     0 : 311.4 : 311.4 : False : False :  Reals
    t23 :     0 : 405.4 : 405.4 : False : False :  Reals
    t24 :

In [10]:
instance.pwc.pprint()

pwc : Size=24, Index=t
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     t1 :     0 :   0.0 :  44.1 : False : False :  Reals
    t10 :     0 :   0.0 : 306.9 : False : False :  Reals
    t11 :     0 :   0.0 : 424.1 : False : False :  Reals
    t12 :     0 :   0.0 :   398 : False : False :  Reals
    t13 :     0 :   0.0 : 487.6 : False : False :  Reals
    t14 :     0 :   0.0 : 521.9 : False : False :  Reals
    t15 :     0 :   0.0 : 541.3 : False : False :  Reals
    t16 :     0 :   0.0 :   560 : False : False :  Reals
    t17 :     0 :   0.0 : 486.8 : False : False :  Reals
    t18 :     0 :   0.0 : 372.6 : False : False :  Reals
    t19 :     0 :   0.0 : 367.4 : False : False :  Reals
     t2 :     0 :   0.0 :  48.5 : False : False :  Reals
    t20 :     0 :   0.0 : 314.3 : False : False :  Reals
    t21 :     0 :   0.0 : 316.6 : False : False :  Reals
    t22 :     0 :   0.0 : 311.4 : False : False :  Reals
    t23 :     0 :   0.0 : 405.4 : False : False :  Reals
    t24 

In [11]:
instance.soc.pprint()

soc : Size=24, Index=t
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
     t1 :  20.0 :  73.33333317313725 :   300 : False : False :  Reals
    t10 :  20.0 :               20.0 :   300 : False : False :  Reals
    t11 :  20.0 :               20.0 :   300 : False : False :  Reals
    t12 :  20.0 :               20.0 :   300 : False : False :  Reals
    t13 :  20.0 :               20.0 :   300 : False : False :  Reals
    t14 :  20.0 :   77.0000003639873 :   300 : False : False :  Reals
    t15 :  20.0 :  134.0000009438783 :   300 : False : False :  Reals
    t16 :  20.0 :  191.0000015230233 :   300 : False : False :  Reals
    t17 :  20.0 : 202.83448620410428 :   300 : False : False :  Reals
    t18 :  20.0 : 136.16781886434794 :   300 : False : False :  Reals
    t19 :  20.0 :  86.66666714591858 :   300 : False : False :  Reals
     t2 :  20.0 :  29.33333323595483 :   300 : False : False :  Reals
    t20 :  20.0 :               20.0 :   300 : False : False :  Rea

In [12]:
instance.pd.pprint()

pd : Size=24, Index=t
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
     t1 :     0 :  24.00000013592513 :  60.0 : False : False :  Reals
    t10 :     0 : 23.074993580450023 :  60.0 : False : False :  Reals
    t11 :     0 :                0.0 :  60.0 : False : False :  Reals
    t12 :     0 :                0.0 :  60.0 : False : False :  Reals
    t13 :     0 :                0.0 :  60.0 : False : False :  Reals
    t14 :     0 :                0.0 :  60.0 : False : False :  Reals
    t15 :     0 :                0.0 :  60.0 : False : False :  Reals
    t16 :     0 :                0.0 :  60.0 : False : False :  Reals
    t17 :     0 :                0.0 :  60.0 : False : False :  Reals
    t18 :     0 :               60.0 :  60.0 : False : False :  Reals
    t19 :     0 : 44.551036538407935 :  60.0 : False : False :  Reals
     t2 :     0 : 39.599999935212836 :  60.0 : False : False :  Reals
    t20 :     0 :               60.0 :  60.0 : False : False :  Real

In [13]:
instance.pc.pprint()

pc : Size=24, Index=t
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
     t1 :     0 :                0.0 :  60.0 : False : False :  Reals
    t10 :     0 :                0.0 :  60.0 : False : False :  Reals
    t11 :     0 :                0.0 :  60.0 : False : False :  Reals
    t12 :     0 :                0.0 :  60.0 : False : False :  Reals
    t13 :     0 :                0.0 :  60.0 : False : False :  Reals
    t14 :     0 :               60.0 :  60.0 : False : False :  Reals
    t15 :     0 :               60.0 :  60.0 : False : False :  Reals
    t16 :     0 :               60.0 :  60.0 : False : False :  Reals
    t17 :     0 : 12.457352284687058 :  60.0 : False : False :  Reals
    t18 :     0 :                0.0 :  60.0 : False : False :  Reals
    t19 :     0 :                0.0 :  60.0 : False : False :  Reals
     t2 :     0 :                0.0 :  60.0 : False : False :  Reals
    t20 :     0 :                0.0 :  60.0 : False : False :  Real